In [ ]:
# import
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
client = chromadb.PersistentClient(path="/home/vuiem/KLTN/chroma_db")
collection = client.create_collection("concept_cards")
google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key="")

collection = client.create_collection(name="kltn_test", embedding_function=google_ef)


In [18]:
import json 
from uuid import uuid4

for week in range(1, 9):

    with open(f'/home/vuiem/KLTN/test/concept_cards/concept_cards_{week}.json') as f:
        data = json.load(f)

    for item in data:
        embedding = google_ef(['\n'.join(item['summary'])])[0]
        collection.add(
            ids=[str(uuid4())],
            embeddings=[embedding],
            documents=['\n'.join(item["summary"])],   # text để search
            metadatas=[{
                "name": item["name"],
                "week": week,
                "formulae": '\n'.join(item["formulae"]),
                "examples": '\n'.join(item["examples"]),
                "common_pitfalls": '\n'.join(item["common_pitfalls"]),
                "page": '\n'.join(map(str, item["page"])),
            }]
        )

In [24]:
query = "Analyze a given dataset scenario, choose an appropriate supervised learning model (e.g., Linear Regression or Logistic Regression), justify the choice, discuss potential issues like overfitting/underfitting, and propose optimization strategies (e.g., regularization)."

embeddings = google_ef([query])[0]
results = collection.query(
    query_embeddings=[embeddings],
    n_results=3,
    where={"week": {"$in": [3, 4, 7]}}
)

results

{'ids': [['9c620f94-b0d6-4ce6-824c-ff1d97ff4b4a',
   '7a88b070-99c7-41a6-a795-7fc7fe074e16',
   'c5aaccea-1217-4e72-9e53-78ad9a0cba6b']],
 'embeddings': None,
 'documents': [['Supervised learning (inductive learning) involves learning a function from labeled training data consisting of input-output pairs.\nThe goal is to approximate an unknown target function f: X -> Y using a hypothesis h: X -> Y, such that h is close to f.\nInput features are denoted as x (a vector in R^n), and outputs are y. For regression, y is a real number (R); for binary classification, y is {+1, -1}; for multi-class classification, y is {1, 2, ..., K}.\nThe training data D is a set of m examples: D = {(x_1, y_1), ..., (x_m, y_m)}.\nThe hypothesis h belongs to a hypothesis space H, which defines the set of possible functions the learning algorithm can choose from (e.g., linear, polynomial, non-linear models).\nA regression problem specifically deals with predicting a real-valued output.',
   "True Error (or Risk

In [20]:
len(results)

8